## Sample notebook showing end-to-end Sales prediction use case using the FedML Databricks Library. 

### The FedML Databricks Library reads the training data via SAP Data Warehouse Cloud, trains the model in Databricks, deploys the model in SAP Kyma Kubernetes and the inference result is written back to SAP Data Warehouse Cloud.

### Install fedml-databricks library

In [ ]:
pip install fedml-databricks --no-cache-dir --upgrade --force-reinstall

In [ ]:
import numpy as np
import pandas as pd
import json
from fedml_databricks import DbConnection,predict

### 1. Connect to SAP Data Warehouse Cloud , Explore & Acquire Data

#### 1.1 Create a Databricks Secret to store the SAP Data Warehouse Cloud connection credentials securely and connect to SAP Data Warehouse Cloud.

Create a Databricks Secret Scope by referring the [(link)](https://docs.databricks.com/security/secrets/secret-scopes.html#create-a-databricks-backed-secret-scope). Then, create the Databricks Secret containing SAP DWC credentials in the form of json, using the [(link)](https://docs.databricks.com/security/secrets/secrets.html#create-a-secret-in-a-databricks-backed-scope). The  SAP DWC connection credentials can be obtained by completing the pre-requisite step using the [(link)](https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Databricks/docs/dbconnection.md#pre-requisite)

In [ ]:
config_str=dbutils.secrets.get('<databriks-secret-scope>','<databricks-secret-key>')
config=json.loads(config_str)

In [ ]:
dwc = DbConnection(dict_obj=config)

#### 1.2 List all business models available to read from

In [ ]:
data= dwc.get_schema_views()
data

Out[4]: ([('Country_Location', 'ROW'),
 ('PP_Gross_Sales_S4', 'ROW'),
 ('ProjectedSalesVolume_View', 'ROW'),
 ('SAP.TIME.VIEW_DIMENSION_DAY', 'ROW'),
 ('SAP.TIME.VIEW_DIMENSION_MONTH', 'ROW'),
 ('SAP.TIME.VIEW_DIMENSION_QUARTER', 'ROW'),
 ('SAP.TIME.VIEW_DIMENSION_YEAR', 'ROW'),
 ('Sales_vs_Outlook_Reporting', 'ROW'),
 ('V_Economic_Outlook__OECD', 'ROW')],
 ['VIEW_NAME', 'VIEW_TYPE'])

#### 1.3 Query the SAP DWC data using SQL Queries. Get the data as a PySpark DataFrame

In [ ]:
spark_df=dwc.execute_query_pyspark('SELECT * FROM \"DEMOSALESANALYSIS\".\"PP_Gross_Sales_S4\"')
spark_df.show(truncate=False)

+-------------+-------------+----------+----+--------------------------------------+----------------------+------------+-----+-------+--------------+-------------+
Country_Label|Country |YEAR_Label|YEAR|HarmonizedCountryDimension_COUNTRYCODE|Projected Sales Volume|Gross amount|Value|Value_1|Gross amount_1|GROSSAMOUNT_1|
+-------------+-------------+----------+----+--------------------------------------+----------------------+------------+-----+-------+--------------+-------------+
United States|United States|2020 |2020|US |104797.12 |104797.12 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2021 |2021|US |104797.12 |104797.12 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2022 |2022|US |104797.12 |104797.12 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2023 |2023|US |104797.12 |104797.12 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2020 |2020|US |34197.75 |34197.75 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2021 |2021|US |34197.75 |34197.75 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2022 |2022|US |34197.75 |34197.75 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2023 |2023|US |34197.75 |34197.75 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2020 |2020|US |46940.60 |46940.60 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2021 |2021|US |46940.60 |46940.60 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2022 |2022|US |46940.60 |46940.60 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2023 |2023|US |46940.60 |46940.60 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2020 |2020|US |87263.99 |87263.99 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2021 |2021|US |87263.99 |87263.99 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2022 |2022|US |87263.99 |87263.99 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2023 |2023|US |87263.99 |87263.99 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2020 |2020|US |92230.86 |92230.86 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2021 |2021|US |92230.86 |92230.86 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2022 |2022|US |92230.86 |92230.86 |0E-10|0E-10 |0.00 |0.00 |
United States|United States|2023 |2023|US |92230.86 |92230.86 |0E-10|0E-10 |0.00 |0.00 |
+-------------+-------------+----------+----+--------------------------------------+----------------------+------------+-----+-------+--------------+-------------+
only showing top 20 rows

##### 1.3.1 Get Insights from the data. In the below cell, we get the average projected sales for the year '2021'

In [ ]:
average_sales_for_2021_df=spark_df.filter(spark_df['YEAR_Label']=='2021').groupBy().avg('Projected Sales Volume')
average_sales_for_2021_df.show(truncate=False)

+---------------------------+
avg(Projected Sales Volume)|
+---------------------------+
72417.156321 |
+---------------------------+

##### 1.3.2 Convert the PySpark DataFrame to Pandas DataFrame

In [ ]:
dataframe=spark_df.toPandas()

/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [Projected Sales Volume, Gross amount, Value, Value_1, Gross amount_1, GROSSAMOUNT_1] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(

#### 1.4 Preprocess the data

##### 1.4.1 Replace the zero values with the mean values in few of the selected columns

In [ ]:
dataframe=dataframe.replace({'GROSSAMOUNT_1': {0: dataframe['GROSSAMOUNT_1'].mean(skipna=True)}}) 
dataframe=dataframe.replace({'Gross amount': {0: dataframe['Gross amount'].mean(skipna=True)}}) 
dataframe=dataframe.replace({'Projected Sales Volume': {0: dataframe['Projected Sales Volume'].mean(skipna=True)}}) 

##### 1.4.2 Perform One Hot Encoding on the Categorical columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder
def one_hot_encode(df,column):
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(df[[column]]).toarray())
    encoded_columns = encoder.get_feature_names([column])
    encoder_df.columns = encoded_columns
    return encoder_df,encoded_columns

In [ ]:
encoded_column_names=[]
for column in ['Country','YEAR']:
    encoded_df,encoded_columns=one_hot_encode(dataframe,column)
    dataframe = dataframe.join(encoded_df)
    encoded_column_names += encoded_columns.tolist()

In [ ]:
dataframe.head(10)

Out[11]:

,Country_Label,Country,YEAR_Label,YEAR,HarmonizedCountryDimension_COUNTRYCODE,Projected Sales Volume,Gross amount,Value,Value_1,Gross amount_1,GROSSAMOUNT_1,Country_Australia,Country_Canada,Country_France,Country_Germany,Country_India,Country_United Kingdom,Country_United States,YEAR_2020,YEAR_2021,YEAR_2022,YEAR_2023
0,United States,United States,2020,2020,US,104797.12,104797.12,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,United States,United States,2021,2021,US,104797.12,104797.12,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,United States,United States,2022,2022,US,104797.12,104797.12,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,United States,United States,2023,2023,US,104797.12,104797.12,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,United States,United States,2020,2020,US,34197.75,34197.75,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5,United States,United States,2021,2021,US,34197.75,34197.75,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
6,United States,United States,2022,2022,US,34197.75,34197.75,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7,United States,United States,2023,2023,US,34197.75,34197.75,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
8,United States,United States,2020,2020,US,46940.60,46940.60,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9,United States,United States,2021,2021,US,46940.60,46940.60,0E-10,0E-10,0.00,14536.153063,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


### 2 Now, using the data,  train the model

In [ ]:
import os,json
import pandas as pd
import mlflow 
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split

label_column = 'Projected Sales Volume'
y = dataframe[label_column]
dataframe.drop(label_column, axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(dataframe , y, test_size=0.3)

#### 2.1 Use the columns required for training the model

In [ ]:
train_columns=['GROSSAMOUNT_1','Gross amount']+encoded_column_names
X_train_dataframe,X_test_dataframe=X_train[train_columns],X_test[train_columns]

#### 2.2 Train the model and log results using mlflow

In [ ]:
def train_model(X_train,X_test, y_train, y_test,experiment_name,model_name):
    mlflow.set_experiment(experiment_name) 
    print("Training model...")

    #Train the LinearRegression model using the fit method
    with mlflow.start_run() as run:
        model = LinearRegression().fit(X_train_dataframe, y_train)
        score = model.score(X_test_dataframe, y_test)
        mlflow.log_param("score",score)
        mlflow.sklearn.log_model(model,model_name,
                         registered_model_name = model_name)
        
    run_id = run.info.run_id
    return run_id


In [ ]:
experiment_name,model_name='/Users/<user>/SalesPredictionExperiment','SalesPredictionModel'
run_id=train_model(X_train,X_test, y_train, y_test,experiment_name,model_name)

Training model...
Registered model 'SalesPredictionModel' already exists. Creating a new version of this model...
2022/08/30 00:51:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: SalesPredictionModel, version 26
Created version '26' of model 'SalesPredictionModel'.

#### 2.3 Take a note of the 'DATABRICKS_URL' and 'MODEL_URI' which needs to be specified in the Databricks config file for SAP Kyma Kubernetes deployment.

In [ ]:
model_uri=f"runs:/{run_id}/{model_name}"
print("The DATABRICKS_URL is 'https://{}'".format(spark.conf.get("spark.databricks.workspaceUrl")))
print("The MODEL_URI is '{}'".format(model_uri))

#### 2.4 Deploy the Databricks MLflow model to SAP Kyma Kubernetes using any of the following Hyperscaler Container Registries:
1. Using Azure Container Registry - Follow this [(notebook)](SAP-Kyma-deploy-using-Azure-ACR/Deploy-Databricks-Model-to-SAP-Kyma-Kubernetes-Using-Azure.ipynb)
2. Using AWS Elastic Container Registry - Follow this [(notebook)](SAP-Kyma-deploy-using-AWS-ECR/Deploy-Databricks-Model-to-SAP-Kyma-Kubernetes-Using-AWS.ipynb)

### 3. Inference the deployed model by passing the test data

In [ ]:
X_test_dataframe['GROSSAMOUNT_1'] = X_test_dataframe['GROSSAMOUNT_1'].astype(float)
X_test_dataframe['Gross amount'] = X_test_dataframe['Gross amount'].astype(float)

<command-585373851679657>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 X_test_dataframe['GROSSAMOUNT_1'] = X_test_dataframe['GROSSAMOUNT_1'].astype(float)
<command-585373851679657>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 X_test_dataframe['Gross amount'] = X_test_dataframe['Gross amount'].astype(float)

In [ ]:
test_data={"data": X_test_dataframe.values.tolist()}

In [ ]:
X_test_dataframe

Out[20]:

,GROSSAMOUNT_1,Gross amount,Country_Australia,Country_Canada,Country_France,Country_Germany,Country_India,Country_United Kingdom,Country_United States,YEAR_2020,YEAR_2021,YEAR_2022,YEAR_2023
775,14536.153063,96891.750000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1721,30808.120000,58144.612252,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
450,14536.153063,209955.360000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1152,14536.153063,8591.620000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1275,14536.153063,117985.490000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,14536.153063,5393.250000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1810,104785.860000,58144.612252,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1256,14536.153063,19079.990000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
683,14536.153063,37308.370000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


#### 3.1 Replace the endpoint_url with the SAP Kyma Kubernetes endpoint url obtained from the deployment notebook.

In [ ]:
import pandas as pd
from fedml_databricks import predict
endpoint_url='<SAP Kyma Kuberenetes endpoint url>'
inference_dataframe=predict(endpoint_url=endpoint_url,data=test_data)
inference_dataframe

Out[21]:

,prediction_result
0,96891.750000
1,58144.612252
2,209955.360000
3,8591.620000
4,117985.490000
...,...
572,5393.250000
573,58144.612252
574,19079.990000
575,37308.370000


### 4 Store the inferencing result in SAP Data Warehouse Cloud

#### 4.1 Store the inference result in the pandas dataframe

In [ ]:
X_test['PredictedSalesVolume']=inference_dataframe['prediction_result'].values

<command-585373851679645>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 X_test['PredictedSalesVolume']=inference_dataframe['prediction_result'].values

#### 4.2 Select the required columns from pandas dataframe

In [ ]:
dwc_write_dataframe=X_test[['Country_Label', 'Country','YEAR_Label', 'YEAR','HarmonizedCountryDimension_COUNTRYCODE','Gross amount', 'Value', 'Value_1', 'Gross amount_1', 'GROSSAMOUNT_1','PredictedSalesVolume']]

#### 4.3 Renaming the columns in the pandas dataframe

In [ ]:
dwc_write_dataframe.rename(columns = {'Gross amount':'Gross_amount', 'Gross amount_1':'Gross_amount_1'}, inplace = True)

<command-585373851679649>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 dwc_write_dataframe.rename(columns = {'Gross amount':'Gross_amount', 'Gross amount_1':'Gross_amount_1'}, inplace = True)

In [ ]:
dwc_write_dataframe

Out[25]:

,Country_Label,Country,YEAR_Label,YEAR,HarmonizedCountryDimension_COUNTRYCODE,Gross_amount,Value,Value_1,Gross_amount_1,GROSSAMOUNT_1,PredictedSalesVolume
775,Australia,Australia,2023,2023,AU,96891.75,0E-10,0E-10,0.00,14536.153063,96891.750000
1721,Australia,Australia,2020,2020,AU,58144.612252,0E-10,0E-10,30808.12,30808.12,58144.612252
450,Germany,Germany,2022,2022,DE,209955.36,0E-10,0E-10,0.00,14536.153063,209955.360000
1152,Canada,Canada,2020,2020,CA,8591.62,0E-10,0E-10,0.00,14536.153063,8591.620000
1275,Canada,Canada,2023,2023,CA,117985.49,0E-10,0E-10,0.00,14536.153063,117985.490000
...,...,...,...,...,...,...,...,...,...,...,...
56,United States,United States,2020,2020,US,5393.25,0E-10,0E-10,0.00,14536.153063,5393.250000
1810,India,India,2020,2020,IN,58144.612252,0E-10,0E-10,104785.86,104785.86,58144.612252
1256,Canada,Canada,2020,2020,CA,19079.99,0E-10,0E-10,0.00,14536.153063,19079.990000
683,United Kingdom,United Kingdom,2023,2023,GB,37308.37,0E-10,0E-10,0.00,14536.153063,37308.370000


In [ ]:
dwc.drop_table('SALES_TABLE')

2022-08-30 00:59:06,322: fedml_databricks.logger INFO: deleting table...

#### 4.4 Create a table in Data Warehouse Cloud for storing the inference result

In [ ]:
dwc.create_table("CREATE TABLE SALES_TABLE (Country_Label Varchar(20),Country Varchar(20),YEAR_Label Varchar(20),YEAR Varchar(20),HarmonizedCountryDimension_COUNTRYCODE Varchar(20), Gross_amount FLOAT,Value FLOAT,Value_1 FLOAT,Gross_amount_1 FLOAT,GROSSAMOUNT_1 FLOAT,PredictedSalesVolume FLOAT)")

2022-08-30 00:59:08,703: fedml_databricks.logger INFO: creating table...
2022-08-30 00:59:08,703: fedml_databricks.logger INFO: CREATE TABLE SALES_TABLE (Country_Label Varchar(20),Country Varchar(20),YEAR_Label Varchar(20),YEAR Varchar(20),HarmonizedCountryDimension_COUNTRYCODE Varchar(20), Gross_amount FLOAT,Value FLOAT,Value_1 FLOAT,Gross_amount_1 FLOAT,GROSSAMOUNT_1 FLOAT,PredictedSalesVolume FLOAT, INSERTED_AT TIMESTAMP NOT NULL)

#### 4.5 Write the prediction results to 'SALES_TABLE' table in SAP Data Warehouse Cloud

In [ ]:
dwc.insert_into_table('SALES_TABLE',dwc_write_dataframe)

2022-08-30 00:59:11,390: fedml_databricks.logger INFO: inserting into table...
2022-08-30 00:59:11,390: fedml_databricks.logger INFO: INSERT INTO SALES_TABLE (Country_Label, Country, YEAR_Label, YEAR, HarmonizedCountryDimension_COUNTRYCODE, Gross_amount, Value, Value_1, Gross_amount_1, GROSSAMOUNT_1, PredictedSalesVolume, INSERTED_AT) VALUES (:Country_Label, :Country, :YEAR_Label, :YEAR, :HarmonizedCountryDimension_COUNTRYCODE, :Gross_amount, :Value, :Value_1, :Gross_amount_1, :GROSSAMOUNT_1, :PredictedSalesVolume, :INSERTED_AT)